# Transformers by HuggingFace 🤗
In this module, we will get familiar with HuggingFace’s `transformers` library. We will first set up `transformers`  package, and then check out some existing pretrained models. We will explore the magical world of the latest chatbot models, sentiment analyzers, and summarizers!

__What you will learn:__

As an ML practitioner, you will often want to explore existing models before developing your own models. By the end of this notebook, you will learn how to use existing models for your tasks using the `pipeline` API and `AutoClass` features. 

The goal of this notebook is not to get familiar with how the library is implemented. Therefore, it is okay if you don't fully understand some details. 

This notebook covers following topics:
- `transformers` library
- ProphetNet
- GPT-2


#### Before we start...

__transformer architecture:__

It would be good to review transformers architecture basics. The notebook assumes you have some basic understanding of what transformers are, and the theory is beyond the scope of this tutorial. 

__transformer library:__

To get an idea about the how the `transformers` library is laid out, do check the [transformers](https://github.com/huggingface/transformers/) repo out . That's where the latest code  `transformers` code exists!

__HuggingFace models:__

Do checkout all models supported by the `transformers` [library](https://huggingface.co/models). We will be using models from this throughout the notebook.


__setup:__

Alright, let's go ahead and install the `transformers` library

In [ ]:
!pip --quiet install transformers

In [1]:
#recommended to have PyTorch, TensorFlow >= 2.0, or Flax installed
import transformers

# Pipeline
Sometimes, we might just want to access off-the-shelf NLP tools like Question Answering modules, sentiment analyzers etc. Transformers package makes it easy to access the existing models through `pipeline`.

In [2]:
from transformers import pipeline

In [3]:
classifier = pipeline("sentiment-analysis")

Downloading:   0%|          | 0.00/629 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/268M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

In [4]:
classifier("I am extremely happy today!")

[{'label': 'POSITIVE', 'score': 0.999880313873291}]

In [5]:
classifier("alright, I want to go for a walk")

[{'label': 'POSITIVE', 'score': 0.9961109161376953}]

Notice how in the above example `alright, I want to go for a walk` is considered a `POSITIVE` sentiment. In reality, this sentence should be detected as a `NEUTRAL` sentiment. 

Why is that? Let's dig into the implementation of `pipeline`. All the implementation related details are in the [`transformers/src/transformers/`](https://github.com/huggingface/transformers/tree/master/src/transformers). The `pipeline` implementation is in the `pipelines.py` file. In [this](https://github.com/huggingface/transformers/blob/443f67e887a030d8254eba126e5f2cdb8b70eb63/src/transformers/pipelines.py#L2729) line we see that the default sentimental analysis model is trained on [SST-2](https://paperswithcode.com/sota/sentiment-analysis-on-sst-2-binary) a binary sentiment classification dataset, and therefore the labels are either `POSITIVE` or `NEGATIVE`. 

As a Machine Learning practitioner, you might want to tweak this model to fit to your own data distribution. In the subsequent modules of this course, we will learn more about pretraining and fine-tuning existing models; in particular, we will use `BERT` based models as it is the most popular NLP model in the last few years.

# HuggingFace Models

There's one really cool place to find all the latest models that are supported by `transformers`. It's the [huggingface models](https://huggingface.co/models) repo!. Let's get the `GPT-2` model, the predecessor of `GPT-3` and do some poetry generation! 🎶

Since the task is text generation, we will pass `text-generation` as the pipeline in our model and use `gpt2` model for generation.  You can safely ignore the warning messages.

In [6]:
from transformers import pipeline, set_seed

generator = pipeline("text-generation", model="gpt2")
set_seed(1234)

my_poem = """
  something in you wakes up
  in the light of the day
  and in the quite of the night
  and I wonder, where did the years go?
  I try to find them in the
  chasms of your breath
  and in the crevices of your broken voice
"""
generated_poem = generator(
    my_poem,
    max_length=200
)[0]["generated_text"]

print(generated_poem)

Downloading:   0%|          | 0.00/665 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/548M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



  something in you wakes up
  in the light of the day
  and in the quite of the night
  and I wonder, where did the years go?
  I try to find them in the
  chasms of your breath
  and in the crevices of your broken voice
  and you are on the threshold of time
  to escape this world  and you know that as long as you continue,
    
     you will ever have me.
     
The   word 'it' is often given from a   -  my  mouth.  It is used as a     reflex.  If you find yourself in a situation in which  you can no longer say  my name, you need not worry, you  can never say my name again.  But, if someone ever   has a case of a memory of yours 


Also, try replacing `gpt2` with the following models and compare the results with different models:
- `lvwerra/gpt2-imdb`
- `sshleifer/tiny-gpt2`

`pipeline` supports a whole slew of tasks some of the following tasks:
- Named Entity Recognition 
- Question-answering
- Summarization
- Translation
- Text Generation
- Text2Text Generation
- Zero-shot classification, and
- Conversational



## Summarization

Alright, let's move on to another interesting task. I often find it so cumbersome to go through giganitic emails or a huge news article. Thankfully for us, there are models that summarize at human-level competency. 

To this end, we will use another `transformer` based model called `ProphetNet` to generate summaries. At the moment of writing this notebook, `ProphetNet` happens to be the state-of-the-art model for generating abstractive summaries.  You can read more about the model in [this](https://arxiv.org/pdf/2001.04063.pdf) paper. 

From here onwards, we will avoid using `pipeline` and get more familiar with some of underlying implementation.

Okay, first things first, find that long email or a text you have been meaning to read for a long time and paste it below:

In [7]:
TEXT_TO_SUMMARIZE = """
copy-paste-your-text-here
"""
# if your text is relatively shorter,
# choose a smaller summary length
summary_len = 20

In [8]:
from transformers import (
    ProphetNetTokenizer,
    ProphetNetForConditionalGeneration,
    ProphetNetConfig
)

# download and load the pre-trained model
model = ProphetNetForConditionalGeneration.from_pretrained(
    'microsoft/prophetnet-large-uncased-cnndm'
)

# download and load the tokenizer
tokenizer = ProphetNetTokenizer.from_pretrained(
    'microsoft/prophetnet-large-uncased-cnndm'
)



Downloading:   0%|          | 0.00/1.24k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.57G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/256 [00:00<?, ?B/s]

In [10]:
# tokenize the input text
inputs = tokenizer([TEXT_TO_SUMMARIZE], max_length=1000, return_tensors='pt', truncation=True)

# Generate Summary
summary_ids = model.generate(
    inputs['input_ids'],
    num_beams=4,
    max_length=summary_len,
    early_stopping=True
)

tokenizer.batch_decode(summary_ids, skip_special_tokens=True)

['...................']

A few things to note here:
- you are not expected to understand the workings of these models. For now, you should consider these models as a black-box and get comfortable with just using existing models in HuggingFace's model repository. Go to [HuggingFace models](https://huggingface.co/models) and replace `microsoft/prophetnet-large-uncased-cnndm` and other summarization models.
- Observe that our summarizer uses a `model` and a `tokenizer`. This is a common pattern for any projects built with transformers as we will see in our next lesson.

In the subsequent notebooks, we will get familiar with `transformers` library. We will get to know the library better by addressing sentence classification and token classification taks.Text generation and summarization models that we have seen in this notebook will not be explored further as they are beyond the scope of this learning package.

# Homework Time! ☕️

Facebook recently released a great open-domain chatbot called Blenderbot. You can read the paper [here](https://arxiv.org/pdf/2004.13637.pdf). The exciting news is that a HuggingFace model exists for Blenderbot! 

The homework for this module is to use Blenderbot to create a small-talk chatbot! 

Here is your [hint](https://huggingface.co/transformers/master/model_doc/blenderbot.html)!

__Note regarding the exercise:__

Since this exercise was first written, the tokenizers corresponding to Blenderbot have been updated, you might run into the following error: `TypeError: forward() got an unexpected keyword argument 'token_type_ids'`. To resolve it simply, remove the `token_type_ids` from the input:
```python
inputs.pop("token_type_ids")
```

## Additional Resources
 - Here is a great [video](https://www.youtube.com/watch?v=KMY2Knr4iAs_) covering the implementational aspects of transformers with a code walk through
 - [Attention is All You Need](https://papers.nips.cc/paper/2017/file/3f5ee243547dee91fbd053c1c4a845aa-Paper.pdf)---the paper that introduced the transformer architecture
